In [2]:
import pandas as pd
import numpy as np
import math
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans 
from sklearn import preprocessing, svm, model_selection

In [ ]:
path = "C:/Users/ozgur/Downloads/fantasy-basketball-master/data/Boxscores/2018-19"
all_files = glob.glob(path + "/*.csv")

In [ ]:
temp_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    temp_list.append(df)

dataframe = pd.concat(temp_list, axis=0, ignore_index=True)

In [ ]:
print(dataframe)

In [ ]:
missing_data = dataframe.isnull()

for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")


In [ ]:
for index, row in dataframe.iterrows():
    if row['3PA'] is 0 and row.isnull()['3P_perc']:
        dataframe.loc[index, ['3P_perc']] = 0

In [ ]:
dataframe.fillna(0, inplace = True)
dataframe.to_csv("merged.csv", encoding='utf-8', index=False)

In [ ]:
data = np.load('C:\\Users\\ozgur\\Downloads\\fantasy-basketball-master\\data\\Names\\standard_names.npy',allow_pickle=True)
print(data)

In [3]:
merged_path = "C:/Users/ozgur/Desktop/dev/Fantasy-Basketball-Player-Recomendation-System/ML/data/Merged/df_2018-19.csv"
merged_df = pd.read_csv(merged_path, index_col=None, header=0)
# merged_df.drop(['Pos', 'Date', 'Salary', 'FPTS', 'Value', 'PG', 'SG', 'F', 'C', 'Name', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA'], axis=1, inplace=True)

for index, row in merged_df.iterrows():
    if row['W'] == row['Team']:
        merged_df.loc[index, ['W']] = 1
    else:
        merged_df.loc[index, ['W']] = 0

# merged_df.drop(['Team'], axis=1, inplace=True)
merged_df.drop(['L'], axis=1, inplace=True)
# merged_df.to_csv("merged.csv", encoding='utf-8', index=False)
print(merged_df)

                  Name    Pos      Date  Salary  Starter Team   FPTS  \
0         Terry Rozier  PG/SG  20181016  4500.0        0  BOS  24.50   
1         Kyrie Irving     PG  20181016  7600.0        1  BOS  21.50   
2         Marcus Smart  PG/SG  20181016  4400.0        0  BOS  15.25   
3       Brad Wanamaker     PG  20181016  3000.0        0  BOS   3.25   
4         Jayson Tatum  SF/PF  20181016  5600.0        1  BOS  40.75   
...                ...    ...       ...     ...      ...  ...    ...   
28906    Royce O'Neale     SF  20190410  3600.0        1  UTA  10.75   
28907  Tyler Cavanaugh     PF  20190410  3000.0        0  UTA   5.25   
28908        Ekpe Udoh      C  20190410  3300.0        1  UTA  47.25   
28909     Tony Bradley      C  20190410  3000.0        0  UTA  37.50   
28910   Derrick Favors   PF/C  20190410  6200.0        0  UTA   0.00   

           Value  Home  W  ...  DRB_perc  ORB_perc  BLK_perc  TOV_perc  \
0       5.444444     1  1  ...      29.6       0.0       3.0 

In [4]:
merged_df.Date = merged_df.Date.astype(str).str.slice(0, 4) + '/' + merged_df.Date.astype(str).str.slice(4, 6) + '/' + merged_df.Date.astype(str).str.slice(6, 8)
merged_df.Date = pd.to_datetime(merged_df.Date)
merged_df.to_csv("merged.csv", encoding='utf-8', index=False)

In [5]:
days = (merged_df.loc[11000, ['Date']] - merged_df.loc[56, ['Date']]).dt.days
print(merged_df.loc[11000, ['Date']], merged_df.loc[56, ['Date']], days)

Date    2018-12-19 00:00:00
Name: 11000, dtype: object Date    2018-10-17 00:00:00
Name: 56, dtype: object Date    63
dtype: int64


In [4]:
merged_path = "C:/Users/ozgur/Desktop/dev/Fantasy-Basketball-Player-Recomendation-System/ML/main_merged2.csv"
merged_df = pd.read_csv(merged_path, index_col=None, header=0)
merged_df.columns

Index(['Name', 'Pos', 'Date', 'Starter', 'Team', 'Home', 'W', 'W_PTS', 'L_PTS',
       'MP', 'FG', 'FGA', 'FG_perc', '3P', '3PA', '3P_perc', 'FT', 'FTA',
       'FT_perc', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'USG_perc',
       'DRtg', 'ORtg', 'AST_perc', 'DRB_perc', 'ORB_perc', 'BLK_perc',
       'TOV_perc', 'STL_perc', 'eFG_perc', 'PG', 'SG', 'F', 'C',
       'PTS_AVG Last Month', 'PTS_AVG Last Week', 'AST Last Week', 'PTS_AVG',
       'AST Last Month', 'MP_AVG', 'FG_AVG', 'FGA_AVG', 'FG_perc_AVG',
       'FG_AVG Last Week', 'FGA_AVG Last Week', 'FG_perc_AVG Last Week',
       'AST_AVG', 'FG_AVG Last Month', 'FGA_AVG Last Month',
       'FG_perc_AVG Last Month', 'Opp', 'BLK_AVG', 'BLK Last Week',
       'BLK Last Month', 'TRB_AVG', 'STL_AVG'],
      dtype='object')

In [ ]:
WEEK = 7
MONTH = 30 

merged_df['MP Last Month'] = np.nan
merged_df.Date = pd.to_datetime(merged_df.Date)
temp_df = merged_df.drop(['Pos', 'Starter', 'Team', 'Home', 'W', 'W_PTS', 'L_PTS',
       'FG', 'FGA', 'FG_perc', '3P', '3PA', '3P_perc', 'FT', 'FTA',
       'FT_perc', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'USG_perc',
       'DRtg', 'ORtg', 'AST_perc', 'DRB_perc', 'ORB_perc', 'BLK_perc',
       'TOV_perc', 'STL_perc', 'eFG_perc', 'PG', 'SG', 'F', 'C',
       'PTS_AVG Last Month', 'PTS_AVG Last Week', 'AST Last Week', 'PTS_AVG',
       'AST Last Month', 'MP_AVG', 'FG_AVG', 'FGA_AVG', 'FG_perc_AVG',
       'FG_AVG Last Week', 'FGA_AVG Last Week', 'FG_perc_AVG Last Week',
       'AST_AVG', 'FG_AVG Last Month', 'FGA_AVG Last Month',
       'FG_perc_AVG Last Month', 'Opp', 'BLK_AVG', 'BLK Last Week',
       'BLK Last Month', 'TRB_AVG', 'STL_AVG'], axis=1)

for index_i, row_i in temp_df.iterrows():
    count = 0
    total = 0

    for index_j, row_j in temp_df.iterrows():
        if index_j >= index_i:
            break
        elif (row_i['Name'] == row_j['Name']) and (row_i['Date'] - row_j['Date']).days <= WEEK:
            count += 1
            total += row_j['MP']
     
    if total is 0:
        merged_df.loc[index_i, ['MP Last Month']] = 0
    else:
        merged_df.loc[index_i, ['MP Last Month']] = total/count
    
    print(index_i, total, count, merged_df.loc[index_i, ['MP Last Month']])
    
print(merged_df)

0 0 0 MP Last Month    0
Name: 0, dtype: object
1 0 0 MP Last Month    0
Name: 1, dtype: object
2 0 0 MP Last Month    0
Name: 2, dtype: object
3 0 0 MP Last Month    0
Name: 3, dtype: object
4 0 0 MP Last Month    0
Name: 4, dtype: object
5 0 0 MP Last Month    0
Name: 5, dtype: object
6 0 0 MP Last Month    0
Name: 6, dtype: object
7 0 0 MP Last Month    0
Name: 7, dtype: object
8 0 0 MP Last Month    0
Name: 8, dtype: object
9 0 0 MP Last Month    0
Name: 9, dtype: object
10 0 0 MP Last Month    0
Name: 10, dtype: object
11 0 0 MP Last Month    0
Name: 11, dtype: object
12 0 0 MP Last Month    0
Name: 12, dtype: object
13 0 0 MP Last Month    0
Name: 13, dtype: object
14 0 0 MP Last Month    0
Name: 14, dtype: object
15 0 0 MP Last Month    0
Name: 15, dtype: object
16 0 0 MP Last Month    0
Name: 16, dtype: object
17 0 0 MP Last Month    0
Name: 17, dtype: object
18 0 0 MP Last Month    0
Name: 18, dtype: object
19 0 0 MP Last Month    0
Name: 19, dtype: object
20 0 0 MP Last Month

163 0 0 MP Last Month    0
Name: 163, dtype: object
164 0 0 MP Last Month    0
Name: 164, dtype: object
165 0 0 MP Last Month    0
Name: 165, dtype: object
166 0 0 MP Last Month    0
Name: 166, dtype: object
167 0 0 MP Last Month    0
Name: 167, dtype: object
168 0 0 MP Last Month    0
Name: 168, dtype: object
169 0 0 MP Last Month    0
Name: 169, dtype: object
170 0 0 MP Last Month    0
Name: 170, dtype: object
171 0 0 MP Last Month    0
Name: 171, dtype: object
172 0 0 MP Last Month    0
Name: 172, dtype: object
173 0 0 MP Last Month    0
Name: 173, dtype: object
174 0 0 MP Last Month    0
Name: 174, dtype: object
175 0 0 MP Last Month    0
Name: 175, dtype: object
176 0 0 MP Last Month    0
Name: 176, dtype: object
177 0 0 MP Last Month    0
Name: 177, dtype: object
178 0 0 MP Last Month    0
Name: 178, dtype: object
179 0 0 MP Last Month    0
Name: 179, dtype: object
180 0 0 MP Last Month    0
Name: 180, dtype: object
181 0 0 MP Last Month    0
Name: 181, dtype: object
182 0 0 MP L

322 0 0 MP Last Month    0
Name: 322, dtype: object
323 0 0 MP Last Month    0
Name: 323, dtype: object
324 0 0 MP Last Month    0
Name: 324, dtype: object
325 0 0 MP Last Month    0
Name: 325, dtype: object
326 0 0 MP Last Month    0
Name: 326, dtype: object
327 0 0 MP Last Month    0
Name: 327, dtype: object
328 0 0 MP Last Month    0
Name: 328, dtype: object
329 0 0 MP Last Month    0
Name: 329, dtype: object
330 0 0 MP Last Month    0
Name: 330, dtype: object
331 0 0 MP Last Month    0
Name: 331, dtype: object
332 0 0 MP Last Month    0
Name: 332, dtype: object
333 0 0 MP Last Month    0
Name: 333, dtype: object
334 0 0 MP Last Month    0
Name: 334, dtype: object
335 0 0 MP Last Month    0
Name: 335, dtype: object
336 0 0 MP Last Month    0
Name: 336, dtype: object
337 0 0 MP Last Month    0
Name: 337, dtype: object
338 0 0 MP Last Month    0
Name: 338, dtype: object
339 0 0 MP Last Month    0
Name: 339, dtype: object
340 0 0 MP Last Month    0
Name: 340, dtype: object
341 0 0 MP L

468 25.78 1 MP Last Month    25.78
Name: 468, dtype: object
469 21.48 1 MP Last Month    21.48
Name: 469, dtype: object
470 28.28 1 MP Last Month    28.28
Name: 470, dtype: object
471 3.97 1 MP Last Month    3.97
Name: 471, dtype: object
472 3.85 1 MP Last Month    3.85
Name: 472, dtype: object
473 23.57 1 MP Last Month    23.57
Name: 473, dtype: object
474 24.4 1 MP Last Month    24.4
Name: 474, dtype: object
475 27.72 1 MP Last Month    27.72
Name: 475, dtype: object
476 27.1 1 MP Last Month    27.1
Name: 476, dtype: object
477 0 0 MP Last Month    0
Name: 477, dtype: object
478 28.55 1 MP Last Month    28.55
Name: 478, dtype: object
479 20.37 1 MP Last Month    20.37
Name: 479, dtype: object
480 3.97 1 MP Last Month    3.97
Name: 480, dtype: object
481 19.52 1 MP Last Month    19.52
Name: 481, dtype: object
482 28.2 1 MP Last Month    28.2
Name: 482, dtype: object
483 27.93 1 MP Last Month    27.93
Name: 483, dtype: object
484 0 0 MP Last Month    0
Name: 484, dtype: object
485 0.0 

609 0.0 1 MP Last Month    0
Name: 609, dtype: object
610 35.22 1 MP Last Month    35.22
Name: 610, dtype: object
611 33.22 1 MP Last Month    33.22
Name: 611, dtype: object
612 1.98 1 MP Last Month    1.98
Name: 612, dtype: object
613 6.6 1 MP Last Month    6.6
Name: 613, dtype: object
614 37.27 1 MP Last Month    37.27
Name: 614, dtype: object
615 23.55 1 MP Last Month    23.55
Name: 615, dtype: object
616 0.0 1 MP Last Month    0
Name: 616, dtype: object
617 64.93 2 MP Last Month    32.465
Name: 617, dtype: object
618 46.89 2 MP Last Month    23.445
Name: 618, dtype: object
619 49.739999999999995 2 MP Last Month    24.87
Name: 619, dtype: object
620 0.82 2 MP Last Month    0.41
Name: 620, dtype: object
621 65.71000000000001 2 MP Last Month    32.855
Name: 621, dtype: object
622 46.25 2 MP Last Month    23.125
Name: 622, dtype: object
623 52.55 2 MP Last Month    26.275
Name: 623, dtype: object
624 1.45 2 MP Last Month    0.725
Name: 624, dtype: object
625 1.45 2 MP Last Month    0.7

...

21070 367.50000000000006 12 MP Last Month    30.625
Name: 21070, dtype: object
21071 131.97 12 MP Last Month    10.9975
Name: 21071, dtype: object
21072 19.990000000000002 10 MP Last Month    1.999
Name: 21072, dtype: object
21073 15.92 7 MP Last Month    2.27429
Name: 21073, dtype: object
21074 314.53999999999996 11 MP Last Month    28.5945
Name: 21074, dtype: object
21075 272.13 11 MP Last Month    24.7391
Name: 21075, dtype: object
21076 157.20999999999998 6 MP Last Month    26.2017
Name: 21076, dtype: object
21077 242.26000000000005 7 MP Last Month    34.6086
Name: 21077, dtype: object
21078 284.32 10 MP Last Month    28.432
Name: 21078, dtype: object
21079 52.239999999999995 11 MP Last Month    4.74909
Name: 21079, dtype: object
21080 273.82 10 MP Last Month    27.382
Name: 21080, dtype: object
21081 274.88 11 MP Last Month    24.9891
Name: 21081, dtype: object
21082 114.13000000000001 11 MP Last Month    10.3755
Name: 21082, dtype: object
21083 329.21000000000004 10 MP Last Month

21185 92.07000000000001 9 MP Last Month    10.23
Name: 21185, dtype: object
21186 20.240000000000002 5 MP Last Month    4.048
Name: 21186, dtype: object
21187 249.73999999999998 10 MP Last Month    24.974
Name: 21187, dtype: object
21188 305.52 10 MP Last Month    30.552
Name: 21188, dtype: object
21189 189.65999999999997 11 MP Last Month    17.2418
Name: 21189, dtype: object
21190 313.69000000000005 10 MP Last Month    31.369
Name: 21190, dtype: object
21191 372.73 10 MP Last Month    37.273
Name: 21191, dtype: object
21192 257.37 10 MP Last Month    25.737
Name: 21192, dtype: object
21193 0 0 MP Last Month    0
Name: 21193, dtype: object
21194 315.86999999999995 10 MP Last Month    31.587
Name: 21194, dtype: object
21195 386.51 11 MP Last Month    35.1373
Name: 21195, dtype: object
21196 225.28 11 MP Last Month    20.48
Name: 21196, dtype: object
21197 185.32999999999998 8 MP Last Month    23.1662
Name: 21197, dtype: object
21198 325.82 11 MP Last Month    29.62
Name: 21198, dtype: o

21301 268.43 10 MP Last Month    26.843
Name: 21301, dtype: object
21302 402.22999999999996 11 MP Last Month    36.5664
Name: 21302, dtype: object
21303 202.08 11 MP Last Month    18.3709
Name: 21303, dtype: object
21304 411.04 12 MP Last Month    34.2533
Name: 21304, dtype: object
21305 345.98 10 MP Last Month    34.598
Name: 21305, dtype: object
21306 283.47 9 MP Last Month    31.4967
Name: 21306, dtype: object
21307 225.93 12 MP Last Month    18.8275
Name: 21307, dtype: object
21308 101.93 7 MP Last Month    14.5614
Name: 21308, dtype: object
21309 215.35 12 MP Last Month    17.9458
Name: 21309, dtype: object
21310 145.33999999999997 12 MP Last Month    12.1117
Name: 21310, dtype: object
21311 70.62 10 MP Last Month    7.062
Name: 21311, dtype: object
21312 23.04 10 MP Last Month    2.304
Name: 21312, dtype: object
21313 294.16 8 MP Last Month    36.77
Name: 21313, dtype: object
21314 255.13000000000002 9 MP Last Month    28.3478
Name: 21314, dtype: object
21315 155.96 10 MP Last Mo

21418 149.34 12 MP Last Month    12.445
Name: 21418, dtype: object
21419 19.990000000000002 10 MP Last Month    1.999
Name: 21419, dtype: object
21420 15.92 8 MP Last Month    1.99
Name: 21420, dtype: object
21421 316.12 9 MP Last Month    35.1244
Name: 21421, dtype: object
21422 210.76 11 MP Last Month    19.16
Name: 21422, dtype: object
21423 94.44 9 MP Last Month    10.4933
Name: 21423, dtype: object
21424 16.75 2 MP Last Month    8.375
Name: 21424, dtype: object
21425 231.91 8 MP Last Month    28.9887
Name: 21425, dtype: object
21426 259.17999999999995 10 MP Last Month    25.918
Name: 21426, dtype: object
21427 183.87 10 MP Last Month    18.387
Name: 21427, dtype: object
21428 190.57 10 MP Last Month    19.057
Name: 21428, dtype: object
21429 0.0 1 MP Last Month    0
Name: 21429, dtype: object
21430 336.90000000000003 10 MP Last Month    33.69
Name: 21430, dtype: object
21431 239.56000000000003 9 MP Last Month    26.6178
Name: 21431, dtype: object
21432 291.4 10 MP Last Month    29

21531 311.15999999999997 10 MP Last Month    31.116
Name: 21531, dtype: object
21532 167.23 8 MP Last Month    20.9037
Name: 21532, dtype: object
21533 95.63 3 MP Last Month    31.8767
Name: 21533, dtype: object


In [9]:
merged_df.to_csv("main_merged3.csv", encoding='utf-8', index=False)

In [92]:
merged_path = "C:/Users/ozgur/Desktop/dev/Fantasy-Basketball-Player-Recomendation-System/ML/merged1.csv"
merged_df = pd.read_csv(merged_path, index_col=None, header=0)
temp_df = merged_df.drop(['Pos', 'Date', 'Name', 'Team', 'Value', 'Home', 'W', 'W_PTS', 'L_PTS', 'Salary', 'FPTS'], axis=1)

k_means = KMeans(init = "k-means++", n_clusters = 8, n_init = 100)
k_means.fit(temp_df)

merged_df['POS Type'] = k_means.labels_
merged_df.to_csv("merged2.csv", encoding='utf-8', index=False)